In [39]:
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential, load_model
from keras.layers import Dense

In [44]:
crimes = pd.read_csv('crimes_data.csv')
colums_to_drop = ['DR_NO','Date Rptd','AREA', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Mocodes',
                  'Premis Cd', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc',
                  'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION','Cross Street']
crimes.drop(columns=colums_to_drop, axis=1, inplace=True)

In [45]:
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['VEHICLE - STOLEN','VEHICLE - ATTEMPT STOLEN', 'BIKE - STOLEN',
                                                      'VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, BIKES, ETC)', 'BOAT - STOLEN',
                                                      'BIKE - ATTEMPTED STOLEN'],'Vehicle Theft')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['BATTERY - SIMPLE ASSAULT','BATTERY POLICE (SIMPLE)','BATTERY ON A FIREFIGHTER', 
                                                       'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
                                                       'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER','OTHER ASSAULT',
                                                      'KIDNAPPING', 'KIDNAPPING - GRAND ATTEMPT', 'MANSLAUGHTER, NEGLIGENT',
                                                      'BRANDISH WEAPON', 'DISCHARGE FIREARMS/SHOTS FIRED', 'CRIMINAL HOMICIDE'],'Assault')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['THEFT PLAIN - PETTY ($950 & UNDER)','THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)',
                                                      'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)',
                                                       'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD',
                                                      'SHOPLIFTING - PETTY THEFT ($950 & UNDER)','SHOPLIFTING - PETTY THEFT ($950 & UNDER)',
                                                      'SHOPLIFTING - ATTEMPT','SHOPLIFTING-GRAND THEFT ($950.01 & OVER)','THEFT, PERSON',
                                                      'DOCUMENT FORGERY / STOLEN FELONY','EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)',
                                                      'DISHONEST EMPLOYEE - GRAND THEFT', 'THEFT OF IDENTITY',
                                                      'BUNCO, GRAND THEFT', 'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)', 
                                                      'PICKPOCKET', 'BUNCO, PETTY THEFT', 'THEFT FROM MOTOR VEHICLE - ATTEMPT',
                                                      'THEFT PLAIN - ATTEMPT', 'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER',
                                                      'CREDIT CARDS, FRAUD USE ($950.01 & OVER)', 'THEFT FROM PERSON - ATTEMPT',
                                                      'PURSE SNATCHING', 'CREDIT CARDS, FRAUD USE ($950 & UNDER)',
                                                      'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01', 
                                                       'DOCUMENT WORTHLESS ($200.01 & OVER)', 'DOCUMENT WORTHLESS ($200 & UNDER)',
                                                      'TILL TAP - PETTY ($950 & UNDER)', 'DISHONEST EMPLOYEE - PETTY THEFT',
                                                      'THEFT, COIN MACHINE - PETTY ($950 & UNDER)', 'PURSE SNATCHING - ATTEMPT',
                                                      'TILL TAP - GRAND THEFT ($950.01 & OVER)', 'PETTY THEFT - AUTO REPAIR',
                                                      'GRAND THEFT / INSURANCE FRAUD', 'THEFT, COIN MACHINE - GRAND ($950.01 & OVER)',
                                                      'THEFT, COIN MACHINE - ATTEMPT', 'PICKPOCKET, ATTEMPT', 'GRAND THEFT / AUTO REPAIR',
                                                      'DISHONEST EMPLOYEE ATTEMPTED THEFT', 'CREDIT CARDS, FRAUD USE ($950 & UNDER'],'Theft')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['BURGLARY FROM VEHICLE','BURGLARY', 'BURGLARY, ATTEMPTED',
                                                      'BURGLARY FROM VEHICLE, ATTEMPTED', 'TRESPASSING', 'SHOTS FIRED AT INHABITED DWELLING',
                                                      ],'Burglary')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)',
                                                       'VANDALISM - MISDEAMEANOR ($399 OR UNDER)', 'THROWING OBJECT AT MOVING VEHICLE',
                                                      'SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT',
                                                      'WEAPONS POSSESSION/BOMBING', 'TELEPHONE PROPERTY - DAMAGE'],'Vandalism')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['ROBBERY','ATTEMPTED ROBBERY'],'Robbery')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['INTIMATE PARTNER - SIMPLE ASSAULT','INTIMATE PARTNER - AGGRAVATED ASSAULT',
                                                      'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT', 'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT',
                                                        'CHILD ANNOYING (17YRS & UNDER)','CHILD NEGLECT (SEE 300 W.I.C.)',
                                                      'CHILD STEALING', 'CHILD ABANDONMENT', 
                                                       'INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)',
                                                      'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)'],'Domestic Violence')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['CRIMINAL THREATS - NO WEAPON DISPLAYED','THREATENING PHONE CALLS/LETTERS',
                                                       'STALKING', 'DISTURBING THE PEACE'],'Criminal Threats')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['RAPE, FORCIBLE','RAPE, ATTEMPTED', 'SEXUAL PENETRATION W/FOREIGN OBJECT',
                                                       'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH',
                                                      'ORAL COPULATION', 'CHILD PORNOGRAPHY', 'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE',
                                                      'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS', 'INDECENT EXPOSURE',
                                                      'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ',
                                                      'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD', 'BATTERY WITH SEXUAL CONTACT',
                                                      'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE', 'LEWD CONDUCT',
                                                       'LEWD/LASCIVIOUS ACTS WITH CHILD', 'BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM'],'Sexual Violence')
crimes['Crm Cd Desc'] = crimes['Crm Cd Desc'].replace(['VIOLATION OF RESTRAINING ORDER','VIOLATION OF COURT ORDER', 
                                                      'OTHER MISCELLANEOUS CRIME', 'FAILURE TO DISPERSE', 'LYNCHING' , 
                                                       'CONSPIRACY', 'EXTORTION', 'FALSE IMPRISONMENT', 'PEEPING TOM',
                                                      'FALSE POLICE REPORT', 'PIMPING', 'PANDERING', 
                                                       'CRUELTY TO ANIMALS', 'ILLEGAL DUMPING', 'DISRUPT SCHOOL',
                                                      'BIGAMY', 'BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIMAL', 
                                                      'BLOCKING DOOR INDUCTION CENTER', 'BRIBERY', 'FAILURE TO YIELD',
                                                      'DRUNK ROLL', 'INCITING A RIOT', 'CONTEMPT OF COURT', 'ARSON',
                                                      'RESISTING ARREST', 'VIOLATION OF TEMPORARY RESTRAINING ORDER',
                                                      'UNAUTHORIZED COMPUTER ACCESS', 'BOMB SCARE', 'BUNCO, ATTEMPT', 'RECKLESS DRIVING',
                                                      'PROWLER', 'DRIVING WITHOUT OWNER CONSENT (DWOC)', 'COUNTERFEIT', 'CONTRIBUTING', 
                                                       'LYNCHING - ATTEMPTED', 'DRUGS, TO A MINOR', 'REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)',
                                                      'FIREARMS RESTRAINING ORDER (FIREARMS RO)', 'FIREARMS EMERGENCY PROTECTIVE ORDER (FIREARMS EPO)'],'Other Crimes')

In [15]:
n_to_remove = 80000
theft_indices = crimes[crimes['Crm Cd Desc'] == 'Theft'].index
indices_to_remove = np.random.choice(theft_indices, n_to_remove, replace=False)
crimes.drop(indices_to_remove, inplace=True)
crimes.reset_index(drop=True, inplace=True)

In [46]:
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['.*SCHOOL.*', 'COLLEGE/JUNIOR COLLEGE/UNIVERSITY'],'School', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['.*MTA.*', '.*BUS STOP.*', '.*TRANSPORTATION.*', '.*HARBOR FRWY STATION.*', '.*LA UNION STATION.*', '7TH AND METRO.*', 'MUNICIPAL BUS LINE INCLUDES.*', 'GREYHOUND OR INTERSTATE BUS'],'Public transport facilities', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['.*STORE.*', 'MINI-MART', '.*DIY CENTER.*', 'MARKET', 'SHOPPING MALL (COMMON AREA)', 'SHOPPING MALL.*', 'AUTO REPAIR SHOP', 'TOBACCO SHOP', '.*AUTO DEALERSHIP.*', 'COMPUTER SERVICES.*', 'PAWN SHOP', 'AUTO SALES LOT', 'GUN/SPORTING GOODS'],'Store', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['.*PARKING.*', '.*GARAGE/CARPORT.*', 'VEHICLE STORAGE LOT.*', 'SLIPS.*'],'Parking', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['.*RESTAURANT/FAST FOOD.*','.*BAR/COCKTAIL/NIGHTCLUB.*' ,'.*COFFEE SHOP.*', '.*NIGHT.*', 'DRIVE TH.*'],'Restaurant/Bar', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace('.*PARK.*','Park', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['HOSPITAL','HEALTH SPA/GYM', 'NURSING/CONVALESCENT/RETIREMENT HOME', 'MEDICAL/DENTAL OFFICES', 'MEDICAL MARIJUANA FACILITIES/BUSINESSES', 'NURSERY/FLOWER SHOP', 'VETERINARIAN.*', 'ABORTION CLINIC.*', 'METHADONE CLINIC', 'HOSPICE', 'DAY CARE.*'],'Medical facility', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['.*MULTI-U.*', 'TRANSIENT ENCAMPMENT', 'TRANSITIONAL HOUSING/HALFWAY HOUSE', 'HOTEL', 'MOTEL', 'GROUP HOME', 'APARTMENT/CONDO COMMON LAUNDRY ROOM', 'OTHER RESIDENCE', 'MISSIONS/SHELTERS', 'CONDOMINIUM/TOWNHOUSE', 'PROJECT/TENEMENT/PUBLIC HOUSING', 'FRAT HOUSE/SORORITY/DORMITORY', 'SHORT-TERM VACATION RENTAL', 'FOSTER HOME BOYS OR GIRLS.*'],'Residential building', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['SINGLE FAMILY DWELLING', 'SINGLE RESIDENC.*', 'MOBILE HOME/TRAILERS/CONSTRUCTION.*'],'Residential building', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['STREET','ALLEY', 'SIDEWALK', 'FREEWAY'],'Street', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['STAPLES CENTER.*','OTHER BUSINESS', 'SEWAGE FACILITY/PIPE', 'TRAM/StreetCAR.*', 'DEPT OF DEFENSE FACILITY', 'CHEMICAL STORAGE/MANUFACTURING PLANT', 'HOCKEY RINK/ICE HOCKEY', 'SKATING.*', 'GAS STATION', 'BANK', 'PUBLIC STORAGE', 'AUTOMATED TELLER MACHINE.*', '.*DROP BOX/MONEY.*', 'ESCALATOR.*', 'GARMENT MANUFACTURER', 'OIL REFINERY', 'TOOL.*', 'TUNNEL', 'TV/RADIO/APPLIANCE', 'MOSQUE.*', 'MORTUARY', 'OFFICE BUILDING/OFFICE', 'CYBERSPACE', 'WAREHOUSE', 'POLICE FACILITY', 'CHURCH/CHAPEL.*', 'LAUNDROMAT', 'GOVERNMENT FACILITY.*', 'STORAGE SHED', 'ABANDONED BUILDING ABANDONED HOUSE', 'BEAUTY/BARBER SHOP', 'WEBSITE', 'LIBRARY', 'BALCONY.*', 'DODGER STADIUM', 'THE BEVERLY CENTER', 'HARDWARE/BUILDING SUPPLY', 'THEATRE/MOVIE', 'NAIL SALON', 'THE BEVERLY CONNECTION', 'STUDIO.*', 'POST OFFICE', 'CHECK CASHING.*', 'STUDIO (FILM/PHOTOGRAPHIC/MUSIC)', 'FIRE STATION', 'DETENTION/JAIL FACILITY', 'SYNAGOGUE/TEMPLE', 'COLISEUM', 'HIGH-RISE BUILDING', 'CONVENTION CENTER', 'MASSAGE PARLOR', 'MUSEUM', 'PUBLIC RESTROOM.*', 'STAIRWELL.*', 'ELEVATOR', 'SPORTS ARENA', 'MANUFACTURING COMPANY', 'ENTERTAINMENT.*', 'SWAP MEET', 'VALET', 'CREDIT UNION', 'FACTORY', 'BOWLING Street.*', 'VISION CARE FACILITY.*', 'FINANCE COMPANY', 'TATTOO PARLOR.*', 'SAVINGS & LOAN', 'ARCADE,GAME ROOM.*', 'ENERGY PLANT.*'],'Other inside', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['MUSCLE BEACH','DRIVEWAY', 'HANDBALL COURTS', 'OTHER PREMISE', 'YARD.*', 'PORCH, RESIDENTIAL', 'OTHER/OUTSIDE', 'RETIRED.*', 'MASS GATHERING LOCATION', 'ABATEMENT LOCATION', 'PAY PHONE', 'PEDESTRIAN OVERCROSSING', 'DAM/RESERVOIR', 'CULTURAL SIGNIFICANCE/MONUMENT', 'WATER FACILITY', 'TRASH CAN/TRASH DUMPSTER', 'BASKETBALL COURTS', 'CONSTRUCTION SITE', 'BEACH', 'MAIL BOX', 'THE GROVE', 'PATIO.*', 'CAR WASH', 'VACANT LOT', 'TOW.*', 'RECYCLING CENTER', 'SPORTS VENUE, OTHER', 'GOLF COURSE.*', 'TRAIN TRACKS', 'CLEANER/Other inside', 'UNDERPASS.*', 'RIVER BED.*', 'EQUIPMENT RENTAL', 'CEMETARY.*', 'POOL-PUBLIC.*', 'OTHER PLACE OF WORSHIP', 'TELECOMMUNICATION FACILITY.*'],'Other outside', regex = True)
crimes['Premis Desc'] = crimes['Premis Desc'].replace(['VEHICLE, PASSENGER.*','OTHER INTERSTATE, CHARTER BUS','TRUCK, COMMERICAL', 'AMTRAK TRAIN', 'AIRCRAFT', 'CATERING.*', 'DELIVERY SERVICE.*', 'TAXI', 'METROLINK TRAIN', 'BUS-CHARTER.*', 'OTHER RR TRAIN.*'],'Vehicle', regex = True)

In [47]:
crimes['Vict Descent'] = crimes['Vict Descent'].replace(['O','S', 'L', 'G', 'D', 'U', 'P', 'Z', 'I', 'V', 'J', 'C', 'F', 'K', ],'A',)
crimes['Vict Descent'] = crimes['Vict Descent'].replace(['-','X'], 'H')
crimes['Vict Descent'] = crimes['Vict Descent'].fillna('H')

In [48]:
crimes['Premis Desc'] = crimes['Premis Desc'].fillna('Residential building')

In [49]:
crimes['Vict Sex'] = crimes['Vict Sex'].replace(['H','X', '-'], 'M')
crimes['Vict Sex'] = crimes['Vict Sex'].fillna('M')

In [50]:
O_Age = crimes[crimes['Vict Age'] != 0]
O_Age['Vict Age'].mean()
crimes['Vict Age'] = crimes['Vict Age'].replace([-3,-2,-1, 0], O_Age['Vict Age'].mean())

In [51]:
crimes['DATE OCC'] = pd.to_datetime(crimes['DATE OCC'])
crimes['Day_Occ'] = crimes['DATE OCC'].dt.day
crimes['Month_Occ'] = crimes['DATE OCC'].dt.month
crimes.drop(columns='DATE OCC', axis=1, inplace=True)

C:\Users\shed\AppData\Local\Temp\ipykernel_44856\2213399913.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crimes['DATE OCC'] = pd.to_datetime(crimes['DATE OCC'])


In [52]:
crimes = crimes[crimes['LAT']>0]
crimes = crimes[crimes['LON']<0]

In [53]:
Y = crimes['Crm Cd Desc']
X = crimes.drop('Crm Cd Desc', axis=1)
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

In [54]:
cat_columns = X.select_dtypes(include=['object']).columns.tolist()
one_hot_encoder = OneHotEncoder(sparse_output=False)
encoded = one_hot_encoder.fit_transform(X[cat_columns])
endoded_df = pd.DataFrame(encoded, columns=one_hot_encoder.get_feature_names_out(cat_columns))
X_encoded = pd.concat([X,endoded_df], axis=1)
X_encoded = X_encoded.drop(cat_columns, axis=1)

In [55]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
num_columns = cat_columns = X.select_dtypes(include=['int64', 'int32', 'float64']).columns.tolist()
scaled = min_max_scaler.fit_transform(X[num_columns])
scaled = pd.DataFrame(scaled, columns=min_max_scaler.get_feature_names_out(num_columns))
X = pd.concat([scaled, endoded_df], axis=1)

In [56]:
X.head()

,TIME OCC,Vict Age,LAT,LON,Day_Occ,Month_Occ,AREA NAME_77th Street,AREA NAME_Central,AREA NAME_Devonshire,AREA NAME_Foothill,...,Premis Desc_Other outside,Premis Desc_Park,Premis Desc_Parking,Premis Desc_Public transport facilities,Premis Desc_Residential building,Premis Desc_Restaurant/Bar,Premis Desc_School,Premis Desc_Store,Premis Desc_Street,Premis Desc_Vehicle
0,0.890585,-0.423729,-0.019421,0.443967,-0.533333,-1.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.720950,-0.610169,0.081821,0.613042,-1.000000,-1.000000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.016964,-0.363071,0.078319,0.640765,-0.200000,-0.818182,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.466497,0.254237,0.472143,0.037485,-1.000000,-1.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.648855,-0.508475,0.635466,-0.137837,-1.000000,-1.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

with open('models/X_train.pkl','wb') as f:
    pickle.dump(X_train,f)

with open('models/X_test.pkl','wb') as f:
    pickle.dump(X_test,f)

with open('models/y_train.pkl','wb') as f:
    pickle.dump(y_train,f)

with open('models/y_test.pkl','wb') as f:
    pickle.dump(y_test,f)

In [42]:
rf_base = RandomForestClassifier()
rf_base.fit(X_train, y_train)

RandomForestClassifier()

In [43]:
with open('models/random_forest_base.pkl','wb') as f:
    pickle.dump(rf_base,f)

In [ ]:
random_grid = {
               'criterion': ['gini', 'entropy', 'log_loss'],
               'n_estimators': [50,100,150,200,250,300,350,400],
               'max_features': ['log2', 'sqrt'],
               'max_depth': [10,12,14,16,18,20,22,24,26,28,30],
               'min_samples_split': [2,5,10],
               'min_samples_leaf': [1,2,4]
              }
rfc = RandomForestClassifier()
rf_search = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 50, cv = 2, verbose=2, random_state=42, n_jobs = -1)
rf_search.fit(X_train, y_train)
best_random = rf_random.best_estimator_

In [ ]:
with open('models/random_forest_best.pkl','wb') as f:
    pickle.dump(best_random,f)

In [ ]:
xgb_model_base = xgb.XGBClassifier(device = 'cuda')
xgb_model_base.fit(X_train, y_train)

In [ ]:
with open('models/xgb_model_base.pkl','wb') as f:
    pickle.dump(xgb_model_base,f)

In [ ]:
params_xgb = {
 'learning_rate' : [0.10,0.15,0.20,0.25,0.30, 0.40],
 'max_depth' : [10, 12, 15, 17, 19, 21, 23, 25],
 'min_child_weight' : [5, 7, 9, 12, 15, 20],
 'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
 'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7, 1.0]
}

xgb_model = xgb.XGBClassifier(device = 'cuda')
rs_model=RandomizedSearchCV(xgb_model,param_distributions=params_xgb,n_iter=100,n_jobs=1,cv=2,verbose=3)
rs_model.fit(X_train, y_train)
xgb_model_best = rs_model.best_estimator_

In [ ]:
with open('models/xgb_model_best.pkl','wb') as f:
    pickle.dump(xgb_model_best,f)

In [ ]:
activ = 'relu'

def baseline_model():
    model = Sequential()
    model.add(Dense(30, input_dim = 45, activation = activ))
    model.add(Dense(30, activation = activ))
    model.add(Dense(10, activation = 'softmax')) 

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy', 'AUC','f1_score'])
    return model

In [ ]:
X = X_train.values
Y = y_train

Y = pd.get_dummies(Y)

In [ ]:
base_nn = baseline_model()
base_history = base_nn.fit(X, Y, epochs=50, batch_size=32, verbose =1)

In [ ]:
base_nn.save('models/NNmodel_base.keras')

In [ ]:
activ = 'elu'

def baseline_model_best():
   
    model = Sequential()
    model.add(Dense(30, input_dim = 45, activation = activ))
    model.add(Dense(30, activation = activ))
    model.add(Dense(30, activation = activ))
    model.add(Dense(30, activation = activ))
    model.add(Dense(30, activation = activ))
    model.add(Dense(30, activation = activ))
    model.add(Dense(30, activation = activ))
    model.add(Dense(30, activation = activ))
    model.add(Dense(10, activation = 'softmax')) 

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adamax', metrics = ['accuracy', 'AUC','f1_score'])
    return model

In [ ]:
best_nn = baseline_model_best()
base_history = best_nn.fit(X, Y, epochs=500, batch_size=32, verbose =1)

In [ ]:
best_nn.save('models/NNmodel_best.keras')